In [9]:
# !pip install pandas numpy langchain langgraph tavily-python langchain_community dotenv

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [64 lines of output]
      /usr/lib/python3/dist-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <module>
              File "<pip-setuptools-caller>", line 14, in <module>
              File "/tmp/pip-wheel-8v58iu52/distribute_944c5a78b85f4d67bccfad97fb92267a/setuptools/__init__.py", line 2, in <module>
                from setuptools.extension import Extension, Lib

In [1]:
# importing required libraries
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

from langchain_community.llms import Ollama
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain import hub

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [25]:
# in-built tool
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool= WikipediaQueryRun(api_wrapper=api_wrapper)

In [26]:
wiki_tool.name

'wikipedia'

In [27]:
loader = WebBaseLoader("https://python.langchain.com/")
docs = loader.load()
documnets = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)


In [28]:
vector_db = FAISS.from_documents(documnets, OllamaEmbeddings())
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x746b5c88c160>)

# Tools

- Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:

    - The name of the tool
    - A description of what the tool is
    - JSON schema of what the inputs to the tool are
    - The function to call
    - Whether the result of a tool should be returned directly to the user
    
- The simpler the input to a tool is, the easier it is for an LLM to be able to use it. Many agents will only work with tools that have a single string input.

## Customizing default tools
https://python.langchain.com/v0.1/docs/modules/tools/#customizing-default-tools



In [29]:
# retriver tool
retriever_tool = create_retriever_tool(retriever, "langchain_search", "Search for information about Langchain. For any questions about LangChain, you must use this tool")

In [30]:
retriever_tool.name

'langchain_search'

In [31]:
# Arxiv Tool
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>))

In [32]:
arxiv.name

'arxiv'

In [33]:
# combining all the tools
tools = [wiki_tool, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/shivani/anaconda3/envs/.venv/lib/python3.9/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langchain_search', description='Search for information about Langchain. For any questions about LangChain, you must use this tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x746b739f6550>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vect

# Agents
- By themselves, language models can't take actions - they just output text. 
- A big use case for LangChain is creating agents. 
- Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them. 
- After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In [2]:
# ollama llm
llm = Ollama(model='llama2')

In [37]:
# get the prompt to use directly from hub. We can modify this.
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [38]:
agent = create_openai_tools_agent(llm, tools, prompt)

In [39]:
# Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [41]:
agent_executor.invoke({
    "input": "Tell me about LangChain agents."
})



> Entering new AgentExecutor chain...


## Doc implementation

In [30]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

from langchain_openai import AzureChatOpenAI

In [18]:
load_dotenv()

os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv("ANTHROPIC_API_KEY")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [19]:
# autotrace llm calls
model = AzureChatOpenAI(
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    openai_api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    azure_deployment=os.getenv('AZURE_DEPLOYMENT'),
    )


In [4]:
search = TavilySearchResults(max_results=1)
search_results = search.invoke("What is the weather in Valsad in Gujarat,India?")
search_results

[{'url': 'https://www.accuweather.com/en/in/valsad/188155/june-weather/188155',
  'content': 'Get the monthly weather forecast for Valsad, Gujarat, India, including daily high/low, historical averages, to help you plan ahead.'}]

In [20]:
response = model.invoke([HumanMessage(content="Hi")])
response.content


AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-afe18529-8a91-4330-80e4-63e334b6dfc1-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [27]:
dict(response).keys()

dict_keys(['content', 'additional_kwargs', 'response_metadata', 'type', 'name', 'id', 'example', 'tool_calls', 'invalid_tool_calls', 'usage_metadata'])

In [21]:
tools = [search]

In [23]:
# binding our tool with model to give the language model knoweldge of the tool/s
model_with_tool = model.bind_tools(tools)

In [28]:
response = model_with_tool.invoke([HumanMessage(content="Hi")])

print(f"Content String: {response.content}")
print(f"Tool Call: {response.tool_calls}")

Content String: Hello! How can I assist you today?
Tool Call: []


In [29]:
# call the model that uses tools 
# response = model_with_tool.invoke([HumanMessage(content="What is the weather in New York?")])

# print(f"Content String: {response.content}")
# print(f"Tool Call: {response.tool_calls}")


"""
Output

    Content String: [{'url': 'https://www.accuweather.com/en/in/valsad/188155/june-weather/188155', 'content': 'Get the monthly weather forecast for Valsad, Gujarat, India, including daily high/low, historical averages, to help you plan ahead.'}]
    
    Tool Call: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather san francisco'}, 'id': 'toolu_01VTP7DUvSfgtYxsq9x4EwMp'}]
"""

"\n    Content String: [{'url': 'https://www.accuweather.com/en/in/valsad/188155/june-weather/188155',\n                    'content': 'Get the monthly weather forecast for Valsad, Gujarat, India, including daily high/low, historical averages, to help you plan ahead.'}]\n    Tool Call: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather san francisco'}, 'id': 'toolu_01VTP7DUvSfgtYxsq9x4EwMp'}]\n"

### Create Agent

In [31]:
agent_executor = create_react_agent(model, tools)


In [32]:
response = agent_executor.invoke({'messages': [HumanMessage(content="Hi")]})
response['messages']

[HumanMessage(content='Hi', id='873990e8-16e2-4b05-a5ab-0acd4fc622c9'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 84, 'total_tokens': 94}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-1ca5d3a5-4f08-43e3-954e-37a1e6bf49c5-0', usage_metadata={'input_tokens': 84, 'output_tokens': 10, 'total_tokens': 94})]

### Streaming

In [33]:
for chunk in agent_executor.stream({'messages': [HumanMessage(content="Hi")]}):
    print(chunk, '\n')

{'agent': {'messages': [AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 84, 'total_tokens': 94}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-8231d4ff-7017-4341-b629-0fc047d3fbe7-0', usage_metadata={'input_tokens': 84, 'output_tokens': 10, 'total_tokens': 94})]}} 



In [34]:
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")

In [35]:
agent_executor =  create_react_agent(model, tools, checkpointer=memory)

config = {
    "configurable": {"thread_id": "tutorial"}
}

In [36]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage("hi")]}, config=config):
    print(chunk)
    print()

{'agent': {'messages': [AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 84, 'total_tokens': 94}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-9037d67d-6c69-4a6b-9942-60a6256a0bee-0', usage_metadata={'input_tokens': 84, 'output_tokens': 10, 'total_tokens': 94})]}}



If we want to start a new conversation, all we have to do is change the thread_id used.